# Explore here

In [12]:
# Your code here
import pandas as pd


ds = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/logistic-regression-project-tutorial/main/bank-marketing-campaign-data.csv", sep = ";")
ds.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [2]:
ds.shape

(41188, 21)

In [3]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [4]:
ds.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [6]:
ds.nunique()

age                 78
job                 12
marital              4
education            8
default              3
housing              3
loan                 3
contact              2
month               10
day_of_week          5
duration          1544
campaign            42
pdays               27
previous             8
poutcome             3
emp.var.rate        10
cons.price.idx      26
cons.conf.idx       26
euribor3m          316
nr.employed         11
y                    2
dtype: int64

In [7]:
ds.drop(["age", "marital", "education", "contact", "month", "day_of_week", "duration", "poutcome", "previous", "nr.employed"], axis = 1, inplace = True)
ds.head()

,job,default,housing,loan,campaign,pdays,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,y
0,housemaid,no,no,no,1,999,1.1,93.994,-36.4,4.857,no
1,services,unknown,no,no,1,999,1.1,93.994,-36.4,4.857,no
2,services,no,yes,no,1,999,1.1,93.994,-36.4,4.857,no
3,admin.,no,no,no,1,999,1.1,93.994,-36.4,4.857,no
4,services,no,no,yes,1,999,1.1,93.994,-36.4,4.857,no


In [9]:
ds = ds.drop_duplicates().reset_index(drop = True)
ds.head()

,job,default,housing,loan,campaign,pdays,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,y
0,housemaid,no,no,no,1,999,1.1,93.994,-36.4,4.857,no
1,services,unknown,no,no,1,999,1.1,93.994,-36.4,4.857,no
2,services,no,yes,no,1,999,1.1,93.994,-36.4,4.857,no
3,admin.,no,no,no,1,999,1.1,93.994,-36.4,4.857,no
4,services,no,no,yes,1,999,1.1,93.994,-36.4,4.857,no


In [10]:
ds.describe().T

,count,mean,std,min,25%,50%,75%,max
campaign,18067.0,3.374052,3.703967,1.000,1.000,2.000,4.000,56.000
pdays,18067.0,918.433608,271.125033,0.000,999.000,999.000,999.000,999.000
emp.var.rate,18067.0,-0.307749,1.720561,-3.400,-1.800,-0.100,1.400,1.400
cons.price.idx,18067.0,93.529292,0.648151,92.201,92.893,93.444,93.994,94.767
cons.conf.idx,18067.0,-40.337101,5.186910,-50.800,-42.700,-41.800,-36.400,-26.900
euribor3m,18067.0,3.163255,1.880761,0.634,1.262,4.191,4.960,5.045


In [11]:
ds.isnull().any(axis=1).sum()/len(ds)*100

np.float64(0.0)